In [10]:
import numpy as np
import pandas as pd
from pandas import ExcelWriter
import openpyxl
from pymongo import MongoClient
import argparse
import get_ghsom_dim

# parser = argparse.ArgumentParser(description='manual to this script')
# parser.add_argument('--name', type=str, default = None)
# # parser.add_argument('--index', type=str, default = None)
# args = parser.parse_args()

prefix = 'entropy-wm5-319'
source_path =prefix.replace('-item-seq','')
layers,max_layer,number_of_digits = [12, 12, 9], 3, [2, 2, 1]
# layers,max_layer,number_of_digits = get_ghsom_dim.layers(prefix)
# read source file to get data attribute
# df_source = pd.read_csv('../data/WPG-unclustered.csv', low_memory=False)
df_source = pd.read_csv('../../raw-data/wm5-1.csv', low_memory=False)
df_source['clustered_label'] = np.nan
# df_source = df[['AWU_Avial_ratio', 'Backlog_Avail_ratio']]

# get cluster start flag


def get_cluster_flag(text_file):
    get_cluster_flag = [i for i, x in enumerate(text_file) if x == '$POS_X']
    get_cluster_flag.append(len(text_file)+1)
    return get_cluster_flag


def format_cluster_info_to_dict(unit_file_name, source_data, saved_data_type=None, structure_type=None, parent_name=None, parent_file_position=None, parent_clustered_string=None):
    Groups_info = []
    
    layer_index = 1
    # read .unit file as python list
    unit_file_path = ('../../applications/%s/GHSOM/output/%s/' % (source_path,prefix)) + unit_file_name + '.unit'
    text_file = open(unit_file_path).read().split()
    print(unit_file_path)
    # get file secation flag
    flag = get_cluster_flag(text_file)

    # get layer index
    if 'lvl' in unit_file_name:
        layer_index = int(unit_file_name.split('lvl')[1][0])
    else:
        layer_index = 1
    # get current file map total size
    map_size = int(text_file[text_file.index('$XDIM')+1]) * int(text_file[text_file.index('$YDIM')+1])
    
    print('============map size============')
    print('All map size:',map_size)
    XDIM = text_file[text_file.index('$XDIM')+1]
    YDIM = text_file[text_file.index('$YDIM')+1]
    print('$XDIM : ',text_file[text_file.index('$XDIM')+1])
    print('$YDIM : ',text_file[text_file.index('$YDIM')+1])
    
    # check parent node name
    if parent_name == None:
        parent_name = 'Root'
    else:
        parent_name = str(parent_name) + '-' + str(parent_file_position)

    # create culstered string
    if parent_clustered_string == None:
        parent_clustered_string = '0.'

    # get current and next index of flag (section by section)
    for i, map_index in zip(range(len(flag)-1), range(map_size)):

        # set section index, get current section range in list
        start_index = flag[i]
        end_index = flag[i+1]
        currentSection = text_file[flag[i]:flag[i+1]]
        # print(currentSection)
        
#         POS_X = currentSection[currentSection.index('$POS_X')+1]
#         POS_Y = currentSection[currentSection.index('$POS_Y')+1]
#         print('$POS_X : ',currentSection[currentSection.index('$POS_X')+1])
#         print('$POS_Y : ',currentSection[currentSection.index('$POS_Y')+1])
        
        
        # use POST_X and POST_Y to create group name
        group_position = currentSection[currentSection.index(
            '$POS_X')+1] + currentSection[currentSection.index('$POS_Y')+1]
        
        group_data_index = currentSection[currentSection.index(
            '$MAPPED_VECS')+1:currentSection.index('$MAPPED_VECS_DIST')] if '$MAPPED_VECS' in currentSection else []
        
        # get submap file name ,None if there is no submap
        sub_map_file_name = currentSection[currentSection.index(
            '$URL_MAPPED_SOMS')+1] if '$URL_MAPPED_SOMS' in currentSection else 'None'
        # print('Sub map name:',sub_map_file_name)

        map_index_int = map_index+1
        
        digit = 1
        while True:
            if (map_index_int // 10) != 0:
                digit = digit + 1 
                map_index_int = map_index_int // 10
            else:
                break
        # parent_clustered_string = '0.'
        zero =''
        for i in range(number_of_digits[layer_index-1]-digit):
            zero = zero + '0'
        
        
        # create cluster string by map index
        cluster_string = str(parent_clustered_string) + zero + str(map_index+1)

        # use group_data_index to get ratio static number
        index = np.array(group_data_index, dtype='int64')
        # print(index)
        current_group_source = df_source.iloc[index, :]

        # get rid of -1(no value) -1 (infinite value)
        current_group_statistic_info = current_group_source.describe().to_dict()

        # if children exist then call function again
        if sub_map_file_name != 'None':
            format_cluster_info_to_dict(sub_map_file_name, source_data, saved_data_type, structure_type, unit_file_name, group_position, cluster_string)
            leaf_node = 0
        else:
            leaf_node = 1
            
            # if current cluster is leaf node then insert label
            # df_source.iloc[index, :]['clustered_label'] = cluster_string
            df_source.set_value(index, 'clustered_label', cluster_string)

        # format data base on different visualization ways
        if str(saved_data_type) == 'tree_structure':
            structure = {}
            structure['name'] = unit_file_name + '-' + group_position
            structure['parent'] = parent_name
            if sub_map_file_name != 'None':
                structure['children'] = sub_map_info

        elif str(saved_data_type) == 'result_detail':
            structure = {
                'name': unit_file_name,
                'group_position': group_position,
                'parent_name': parent_name,
                'sub_map_file_name': sub_map_file_name,
                'leaf_node_or_note': leaf_node,
                'statistic_info': current_group_statistic_info,
                'group_data_index': group_data_index,
                'cluster_string': cluster_string
            }

            if str(structure_type) == 'flat':
                # create dictionary for store into mongodb
                Groups_info_flat.append(structure)

    return Groups_info


def get_map_pos(text_file):
    cluster_pos_X_index = [i for i, x in enumerate(text_file) if x == '$POS_X']
    cluster_pos_Y_index = [i for i, x in enumerate(text_file) if x == '$POS_Y']

    pos_X = []
    for i in cluster_pos_X_index:
        pos_X.append(text_file[i+1])

    pos_Y = []
    for i in cluster_pos_Y_index:
        pos_Y.append(text_file[i+1])

    # create column name (as key)
    column_name = []
    for i in range(len(pos_X)):
        name = pos_X[i] + '_' + pos_Y[i]
        column_name.append(name)

    column_position_dict = {'grouop_position': column_name}

    return column_position_dict


# connect to mongoDB
client = MongoClient()
db = client['result']

Groups_info_flat = []
saved_file_type = 'result_detail'
result = format_cluster_info_to_dict(
    prefix, df_source, saved_file_type, 'flat')
# print(Groups_info_flat)

# df_source.to_csv('./applications/%s/data/%s_with_clustered_label.csv' % (source_path, prefix), index=False)
df_source

../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319.unit
============map size============
All map size: 12
$XDIM :  4
$YDIM :  3
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_0x0.unit
============map size============
All map size: 6
$XDIM :  3
$YDIM :  2
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_1x0.unit
============map size============
All map size: 9
$XDIM :  3
$YDIM :  3


C:\Users\soslab\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_2x0.unit
============map size============
All map size: 6
$XDIM :  2
$YDIM :  3
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_3x0.unit
============map size============
All map size: 4
$XDIM :  2
$YDIM :  2
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_0x1.unit
============map size============
All map size: 8
$XDIM :  2
$YDIM :  4
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_1x1.unit
============map size============
All map size: 9
$XDIM :  3
$YDIM :  3
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_2x1.unit
============map size============
All map size: 9
$XDIM :  3
$YDIM :  3
../../applications/entropy-wm5-319/GHSOM/output/entropy-wm5-319/entropy-wm5-319_submap_lvl2_3x1.unit
============map size============
All ma

,id,week,TA_video,TA_text,Teacher_video,Teacher_text,clustered_label
0,100101021,week1,-0.178038,-0.126751,-0.178561,-0.201841,0.10051
1,100101021,week2,-0.178038,-0.126751,-0.175084,-0.143443,0.1008
2,100101021,week3,-0.166362,-0.048546,-0.176629,-0.174506,0.1009
3,100101021,week4,-0.178038,-0.116324,-0.178561,-0.137230,0.1008
4,100101021,week5,-0.178038,-0.012050,-0.178561,-0.189416,0.1009
5,100101021,week6,-0.163860,-0.126751,-0.175470,-0.042177,0.1007
6,100101021,week7,-0.178038,-0.062883,-0.178561,-0.172642,0.1008
7,100101021,week8,-0.178038,-0.126751,-0.178561,-0.173263,0.1008
8,100101021,week9,-0.178038,-0.057670,-0.178561,-0.177612,0.1008
9,100101021,week10,-0.178038,-0.082435,-0.178561,-0.171399,0.1008
